
<br>
Introduction to PydanticAI.<br>
This module demonstrates how PydanticAI makes it easier to build<br>
production-grade LLM-powered systems with type safety and structured responses.<br>


In [77]:
from typing import Dict, List, Optional
#import nest_asyncio
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext, Tool
from pydantic_ai.models.openai import OpenAIModel

In [78]:
from utils.markdown import to_markdown
import nest_asyncio
nest_asyncio.apply()

In [79]:
model = OpenAIModel(base_url="https://openrouter.ai/api/v1", model_name="google/gemini-2.0-flash-exp:free")

In [80]:
class ResponseModel(BaseModel):
    """Structured response with metadata."""
    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")

In [81]:
class Order(BaseModel):
    """Structure for order details."""
    order_id: str
    status: str
    items: List[str]

In [82]:
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries."""
    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] = None

In [83]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
    ],
)

In [84]:
shipping_info_db: Dict[str, str] = {
    "12345": "Shipped on 2024-12-01",
    "67890": "Out for delivery",
}

In [85]:
def get_shipping_info(ctx: RunContext[CustomerDetails]) -> str:
    """Get the customer's shipping information."""
    return shipping_info_db[ctx.deps.orders[0].order_id]

In [86]:
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
    tools=[Tool(get_shipping_info, takes_ctx=True)],  # Add tool via kwarg
)

In [87]:
@agent5.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"

In [88]:
response = agent5.run_sync(
    user_prompt="What's the status of my last order?", deps=customer
)

In [ ]:
response.all_messages()
print(response.data.model_dump_json(indent=2))

----


<br>
This example demonstrates advanced agent capabilities with self-correction.<br>
Key concepts:<br>
- Implementing self-reflection<br>
- Handling errors gracefully with retries<br>
- Using ModelRetry for automatic retries<br>
- Decorator-based tool registration<br>


Simulated database of shipping information

In [90]:
shipping_info_db: Dict[str, str] = {
    "#12345": "Shipped on 2024-12-01",
    "#67890": "Out for delivery",
}

In [91]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
)

Agent with reflection and self-correction

In [92]:
agent5 = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information. "
        "Always greet the customer and provide a helpful response."
    ),
)

In [93]:
@agent5.tool_plain()  # Add plain tool via decorator
def get_shipping_status(order_id: str) -> str:
    """Get the shipping status for a given order ID."""
    shipping_status = shipping_info_db.get(order_id)
    if shipping_status is None:
        raise ModelRetry(
            f"No shipping information found for order ID {order_id}. "
            "Make sure the order ID starts with a #: e.g, #624743 "
            "Self-correct this if needed and try again."
        )
    return shipping_info_db[order_id]

In [94]:
response = agent5.run_sync(
    user_prompt="What's the status of my last order 12345?", deps=customer
)

In [ ]:
response.all_messages()
print(response.data.model_dump_json(indent=2))